In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
ORG_NAME = "Magic Square"
SPLITS_FREQ = "7d"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2023-07-17"


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 316 non-null    object        
 1   author             316 non-null    object        
 2   date               316 non-null    datetime64[us]
 3   start              316 non-null    datetime64[us]
 4   end                316 non-null    datetime64[us]
 5   platform_proposal  316 non-null    object        
dtypes: datetime64[us](3), object(3)
memory usage: 14.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899961 entries, 0 to 899960
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        899961 non-null  object        
 1   proposal  899961 non-null  object        
 2   voter     899961 non-null  object        
 3   date      899961 non-null  datetime64[us]

In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x2212d5f1e531d9786f12df2af0661cfd58cbae71,5611034c-c172-5e23-9c3b-1268fde09c85,2023-05-08 11:56:03,1
1,0x5f8ba595ede4554929a04a0ce9c0d7b31aa3661f,5611034c-c172-5e23-9c3b-1268fde09c85,2023-05-08 11:57:28,1
2,0xef6ad4fcaf7be57f244e617394b5968d3db9ffa0,5611034c-c172-5e23-9c3b-1268fde09c85,2023-05-08 11:57:48,1
3,0xa7db4f21096c45d9392affb9f9ed65110c2838d1,5611034c-c172-5e23-9c3b-1268fde09c85,2023-05-08 12:02:28,1
4,0xe39a6e7d1cfa126b2dcdba9bf8124ba403f37a68,5611034c-c172-5e23-9c3b-1268fde09c85,2023-05-08 12:02:33,1
...,...,...,...,...
899956,0xa607a4b265dd82fc99169faeb29470e2bf3566ce,cdcb5a6f-c0c6-5b1a-840b-85715450033b,2023-07-20 07:53:03,1
899957,0xa607a4b265dd82fc99169faeb29470e2bf3566ce,6b685db4-bf17-5c9a-9214-3ffa835699a3,2023-07-20 07:53:58,1
899958,0xa3d904e8cd2366d27724e0454cb5641cf1380be8,ceb1134c-c268-5d54-9ed6-4edbf129cd87,2023-07-20 08:15:16,1
899959,0xa3d904e8cd2366d27724e0454cb5641cf1380be8,6b685db4-bf17-5c9a-9214-3ffa835699a3,2023-07-20 08:15:59,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2023-05-15 00:00:00'),
 Timestamp('2023-05-22 00:00:00'),
 Timestamp('2023-05-29 00:00:00'),
 Timestamp('2023-06-05 00:00:00'),
 Timestamp('2023-06-12 00:00:00'),
 Timestamp('2023-06-19 00:00:00'),
 Timestamp('2023-06-26 00:00:00'),
 Timestamp('2023-07-03 00:00:00'),
 Timestamp('2023-07-10 00:00:00'),
 Timestamp('2023-07-17 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2023-07-17 00:00:00
time_train                    5.771259
time_rec                      1.704674
open_proposals                      16
min_recs                             1
avg_recs                         4.108
precision@1                     0.7192
precision@3                   0.344533
precision@5                    0.27488
precision@10                   0.22104
precision@15                  0.164373
precision@100                 0.024936
ndcg@1                        0.331979
ndcg@3                        0.342046
ndcg@5                        0.352872
ndcg@10                       0.390243
ndcg@15                       0.395788
ndcg@100                      0.396163
map@1                           0.7192
map@3                         0.734244
map@5                         0.742569
map@10                        0.797308
map@15                        0.804489
map@100                       0.805239
recall@1                       0.60037
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/Magic Square/hparams-itemknn_7d_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-05-15,1,7d,2023-05-15,0.007447,0.253381,13,1,5.737452,0.776062,...,0.969393,1.000000,1.0,0.413684,0.740494,0.847857,0.929851,0.938788,0.938788,0.359887
1,2023-05-15,1,14d,2023-05-15,0.005742,0.249509,13,1,5.737452,0.776062,...,0.969393,1.000000,1.0,0.413684,0.740494,0.847857,0.929851,0.938788,0.938788,0.423466
2,2023-05-15,1,21d,2023-05-15,0.005372,0.249433,13,1,5.737452,0.776062,...,0.969393,1.000000,1.0,0.413684,0.740494,0.847857,0.929851,0.938788,0.938788,0.358904
3,2023-05-15,1,30d,2023-05-15,0.005656,0.249554,13,1,5.737452,0.776062,...,0.969393,1.000000,1.0,0.413684,0.740494,0.847857,0.929851,0.938788,0.938788,0.359795
4,2023-05-15,1,60d,2023-05-15,0.005655,0.252159,13,1,5.737452,0.776062,...,0.969393,1.000000,1.0,0.413684,0.740494,0.847857,0.929851,0.938788,0.938788,0.360428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2023-07-17,15,21d,2023-07-17,0.345537,1.934927,16,1,4.295276,0.741732,...,0.963195,0.996776,1.0,0.805719,0.846358,0.875804,0.913588,0.926067,0.927669,0.779905
766,2023-07-17,15,30d,2023-07-17,0.421903,2.001630,16,1,4.947955,0.689963,...,0.912171,0.978119,1.0,0.811426,0.862458,0.895905,0.936563,0.952870,0.955191,0.805036
767,2023-07-17,15,60d,2023-07-17,0.638957,2.556281,16,1,5.669910,0.651842,...,0.860332,0.964421,1.0,0.800664,0.859264,0.896248,0.946921,0.969978,0.973450,0.743937
768,2023-07-17,15,90d,2023-07-17,0.656193,2.611269,16,1,5.839371,0.659604,...,0.855592,0.964053,1.0,0.781950,0.849424,0.886160,0.934718,0.958546,0.962185,0.836180


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train   avg_recs  precision@5  precision@10  \
window_size k                                                      
7d          8     0.062619  15.296094     0.443229      0.393439   
            9     0.062212  15.296094     0.442976      0.393356   
            10    0.064081  15.296094     0.442199      0.393429   
            7     0.061835  15.296094     0.440302      0.391673   
            15    0.063579  15.296094     0.441077      0.392198   
...                    ...        ...          ...           ...   
90d         2     0.269910  16.498979     0.322060      0.307861   
10YE        2     0.268585  16.498979     0.322060      0.307861   
60d         1     0.268761  16.479449     0.320011      0.306429   
90d         1     0.269533  16.498979     0.320669      0.306895   
10YE        1     0.269399  16.498979     0.320669      0.306895   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          8        0.064895  0.081042  0.090189  0.102231  0.577343   
            9        0.064895  0.080956  0.090185  0.102248  0.576706   
            10       0.064895  0.080805  0.090189  0.102195  0.575539   
            7        0.064895  0.080572  0.089620  0.102103  0.575059   
            15       0.064895  0.080577  0.089679  0.101970  0.574053   
...                       ...       ...       ...       ...       ...   
90d         2        0.070217  0.133761  0.152908  0.219153  0.407161   
10YE        2        0.070217  0.133761  0.152908  0.219153  0.407161   
60d         1        0.070120  0.132290  0.151282  0.217978  0.405370   
90d         1        0.070217  0.133010  0.151968  0.218849  0.405294   
10YE        1        0.070217  0.133010  0.151968  0.218849  0.405294   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          8   0.612354  0.677356  0.501102   0.724208         1.0   
            9   0.612222  0.677272  0.500439   0.723787         1.0   
            10  0.611692  0.676629  0.498573   0.724360         1.0   
            7   0.610364  0.676596  0.498637   0.719851         1.0   
            15  0.609687  0.675030  0.497376   0.720314         1.0   
...                  ...       ...       ...        ...         ...   
90d         2   0.436223  0.541608  0.330359   0.504008         1.0   
10YE        2   0.436223  0.541608  0.330359   0.504008         1.0   
60d         1   0.434688  0.540512  0.329433   0.501887         1.0   
90d         1   0.434373  0.540446  0.328922   0.501328         1.0   
10YE        1   0.434373  0.540446  0.328922   0.501328         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          8        0.620533        0.729446         0.812319  
            9        0.620163        0.730352         0.813183  
            10       0.619492        0.729837         0.812713  
            7        0.617149        0.727347         0.810208  
            15       0.619059        0.729160         0.812036  
...                       ...             ...              ...  
90d         2        0.591395        0.664057         0.780506  
10YE        2        0.591395        0.664057         0.780506  
60d         1        0.588952        0.669037         0.777656  
90d         1        0.587732        0.667901         0.776608  
10YE        1        0.587732        0.667901         0.776608  

[77 rows x 17 columns]

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Magic Square/models/itemknn-best-avg_7d_normalize.parquet


/tmp/ipykernel_483168/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-05-15,7d,8,2023-05-15,0.005447,0.249887,13,1,5.737452,0.772201,0.667954,0.542857,0.364093,...,0.977115,1.000000,1.0,0.416419,0.737283,0.845497,0.927115,0.936011,0.936011,0.359182
2023-05-22,7d,8,2023-05-22,0.008454,0.311174,32,1,20.579288,0.488673,0.470334,0.434951,0.393851,...,0.612292,0.757680,1.0,0.291855,0.447078,0.527575,0.641276,0.716241,0.775718,1.209782
2023-05-29,7d,8,2023-05-29,0.020779,0.695229,31,1,20.691438,0.492730,0.467959,0.448142,0.386753,...,0.670089,0.773453,1.0,0.323133,0.467931,0.557292,0.651025,0.698960,0.736976,2.872979
2023-06-05,7d,8,2023-06-05,0.042552,0.947450,25,1,15.005519,0.541943,0.489330,0.479029,0.460486,...,0.715873,0.858154,1.0,0.327271,0.462272,0.552564,0.698307,0.756016,0.774256,3.704591
2023-06-12,7d,8,2023-06-12,0.048322,1.237559,23,1,14.218667,0.494222,0.436444,0.421156,0.394933,...,0.684940,0.900841,1.0,0.304537,0.457929,0.546867,0.681182,0.737049,0.769926,3.446731
2023-06-19,7d,8,2023-06-19,0.061203,1.128895,19,1,12.411872,0.763470,0.627093,0.532237,0.412603,...,0.860159,0.955765,1.0,0.436030,0.678399,0.761954,0.855131,0.896689,0.911518,2.573428
2023-06-26,7d,8,2023-06-26,0.049711,1.258376,22,1,14.352895,0.368545,0.562598,0.603756,0.489045,...,0.744034,0.967508,1.0,0.272887,0.390413,0.598003,0.698368,0.733240,0.742038,5.311892
2023-07-03,7d,8,2023-07-03,0.172470,1.428532,30,1,18.978957,0.675206,0.528210,0.486551,0.469350,...,0.625048,0.784864,1.0,0.368663,0.479360,0.495469,0.634969,0.736799,0.809398,4.847252
2023-07-10,7d,8,2023-07-10,0.083215,4.011816,33,1,15.097138,0.365162,0.260594,0.270776,0.292075,...,0.722610,0.884627,1.0,0.581186,0.642895,0.692964,0.797518,0.834831,0.854647,8.726313


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Magic Square/models/itemknn-best-val_7d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-05-15,1,7d,0.007447,5.737452,0.545946,0.363320,0.037954,0.231556,0.244176,0.246566,0.801814,0.831815,0.838071,0.798848,0.969393,1.0,0.847857,0.929851,0.938788
2023-05-22,9,7d,0.008410,20.579288,0.436246,0.394822,0.079353,0.075182,0.082105,0.099982,0.508334,0.528781,0.615637,0.397495,0.613114,1.0,0.527174,0.641879,0.775330
2023-05-29,4,7d,0.018302,20.691438,0.453312,0.388368,0.073134,0.049662,0.053310,0.063323,0.539673,0.552715,0.632676,0.482184,0.675014,1.0,0.566762,0.653108,0.738013
2023-06-05,2,14d,0.065472,14.086324,0.538118,0.491659,0.076634,0.072994,0.079641,0.090732,0.604652,0.646563,0.714626,0.463085,0.705779,1.0,0.565347,0.732957,0.825980
2023-06-12,6,14d,0.094112,13.761174,0.466444,0.424550,0.066858,0.109197,0.121668,0.141357,0.538226,0.572705,0.632625,0.426266,0.677451,1.0,0.580444,0.739916,0.831261
2023-06-19,1,7d,0.061567,12.411872,0.533881,0.414521,0.055744,0.156749,0.163917,0.171901,0.792537,0.816631,0.849249,0.696607,0.862996,1.0,0.755141,0.849648,0.905554
2023-06-26,1,7d,0.050541,14.352895,0.601095,0.488028,0.065610,0.055294,0.059444,0.067575,0.588895,0.630819,0.701996,0.504124,0.742492,1.0,0.593065,0.752769,0.796415
2023-07-03,1,7d,0.166977,18.978957,0.566880,0.479506,0.098618,0.088217,0.087418,0.099770,0.730073,0.694111,0.781563,0.480516,0.635961,1.0,0.523824,0.641042,0.824033
2023-07-10,10,7d,0.083697,15.097138,0.270006,0.294662,0.045482,0.094868,0.126545,0.146878,0.394203,0.480399,0.530141,0.382004,0.731235,1.0,0.691758,0.810450,0.867360


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-06-19 00:00:00,0.079563,1.474019,25.666667,1.0,14.896343,0.601781,0.499927,0.463171,0.399773,...,0.733843,0.870650,1.0,0.396999,0.530106,0.626778,0.743358,0.796974,0.826992,3.693127
min,2023-05-22 00:00:00,0.008410,0.380533,16.000000,1.0,4.108000,0.364062,0.259585,0.270006,0.221840,...,0.613114,0.760444,1.0,0.209715,0.385305,0.523824,0.641042,0.700404,0.738013,0.694549
25%,2023-06-05 00:00:00,0.050541,1.122870,22.000000,1.0,13.761174,0.500808,0.471729,0.436246,0.388368,...,0.675014,0.798289,1.0,0.292467,0.445129,0.565347,0.653108,0.751740,0.796415,2.543982
50%,2023-06-19 00:00:00,0.065472,1.259720,25.000000,1.0,14.352895,0.560621,0.493218,0.466444,0.414521,...,0.705779,0.847109,1.0,0.323564,0.459734,0.580444,0.739916,0.797209,0.825980,3.564492
75%,2023-07-03 00:00:00,0.094112,1.631213,31.000000,1.0,18.978957,0.728000,0.567293,0.538118,0.479506,...,0.742492,0.956210,1.0,0.431916,0.642279,0.691758,0.810450,0.847613,0.867360,4.800728
max,2023-07-17 00:00:00,0.166986,4.020334,33.000000,1.0,20.691438,0.782648,0.636780,0.601095,0.491659,...,0.960547,0.997315,1.0,0.828164,0.809121,0.837491,0.868451,0.890847,0.905554,8.673960
std,NaN,0.056671,1.048765,6.123724,0.0,5.091513,0.146140,0.115957,0.114202,0.091025,...,0.111659,0.087102,0.0,0.194311,0.141113,0.109704,0.087011,0.067042,0.052505,2.382745


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Magic Square/models/itemknn-best-test_7d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-05-22,1,7d,0.008328,20.579288,0.413592,0.381877,0.079353,0.070858,0.078216,0.097361,0.470489,0.496298,0.587388,0.380388,0.588375,1.0,0.501116,0.614727,0.755603
2023-05-29,9,7d,0.018340,20.691438,0.448142,0.386591,0.073134,0.049108,0.052864,0.063006,0.531151,0.546826,0.627185,0.476362,0.669628,1.0,0.556745,0.650724,0.736763
2023-06-05,4,7d,0.042704,15.005519,0.494040,0.461810,0.071943,0.061068,0.068547,0.078528,0.564907,0.616789,0.691530,0.462888,0.718242,1.0,0.544158,0.706275,0.782324
2023-06-12,2,14d,0.094775,13.761174,0.464443,0.416411,0.066858,0.109572,0.119866,0.141540,0.541616,0.568932,0.633215,0.422116,0.647430,1.0,0.584897,0.744423,0.835578
2023-06-19,6,14d,0.115288,11.799587,0.514738,0.432094,0.059146,0.184893,0.197782,0.215014,0.708687,0.735498,0.782647,0.592190,0.801862,1.0,0.720176,0.833678,0.889262
2023-06-26,1,7d,0.050541,14.352895,0.601095,0.488028,0.065610,0.055294,0.059444,0.067575,0.588895,0.630819,0.701996,0.504124,0.742492,1.0,0.593065,0.752769,0.796415
2023-07-03,1,7d,0.166977,18.978957,0.566880,0.479506,0.098618,0.088217,0.087418,0.099770,0.730073,0.694111,0.781563,0.480516,0.635961,1.0,0.523824,0.641042,0.824033
2023-07-10,1,7d,0.083514,15.097138,0.205118,0.237012,0.045482,0.082222,0.106498,0.139506,0.345170,0.412670,0.487029,0.330466,0.585634,1.0,0.702221,0.730320,0.791922
2023-07-17,10,7d,0.076376,6.329073,0.311821,0.242173,0.031246,0.088148,0.093211,0.097400,0.735007,0.761456,0.783299,0.739071,0.883896,1.0,0.851325,0.906235,0.935266


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-06-19 00:00:00,0.072983,1.376050,25.666667,1.0,15.177230,0.581568,0.476909,0.446652,0.391722,...,0.697058,0.867534,1.0,0.389054,0.528949,0.619725,0.731133,0.786357,0.816352,3.698269
min,2023-05-22 00:00:00,0.008328,0.381587,16.000000,1.0,6.329073,0.335168,0.212530,0.205118,0.237012,...,0.585634,0.745726,1.0,0.209715,0.385305,0.501116,0.614727,0.694385,0.736763,0.672069
25%,2023-06-05 00:00:00,0.042704,0.615560,22.000000,1.0,13.761174,0.492730,0.439051,0.413592,0.381877,...,0.635961,0.798289,1.0,0.275147,0.448567,0.544158,0.650724,0.751740,0.782324,2.636512
50%,2023-06-19 00:00:00,0.076376,1.259720,25.000000,1.0,15.005519,0.549669,0.494481,0.464443,0.416411,...,0.669628,0.858859,1.0,0.331544,0.463619,0.584897,0.730320,0.771550,0.796415,3.602550
75%,2023-07-03 00:00:00,0.094775,1.575393,31.000000,1.0,18.978957,0.692488,0.567293,0.514738,0.461810,...,0.742492,0.952326,1.0,0.380816,0.623994,0.702221,0.752769,0.801458,0.835578,4.800728
max,2023-07-17 00:00:00,0.166977,4.111504,33.000000,1.0,20.691438,0.767612,0.636780,0.601095,0.488028,...,0.883896,0.994256,1.0,0.754085,0.812386,0.851325,0.906235,0.932390,0.935266,8.648197
std,NaN,0.049760,1.127608,6.123724,0.0,4.561535,0.141818,0.124284,0.124056,0.093912,...,0.099738,0.088670,0.0,0.179245,0.145049,0.114783,0.093923,0.077052,0.063307,2.376082


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2023-07-17 00:00:00
time_train                    0.839565
time_rec                      2.716351
open_proposals                      16
min_recs                             1
avg_recs                         4.108
precision@1                     0.8432
precision@3                   0.507467
precision@5                      0.368
precision@10                   0.22584
precision@15                  0.164853
precision@100                 0.024936
ndcg@1                        0.389217
ndcg@3                        0.414949
ndcg@5                        0.424046
ndcg@10                       0.430003
ndcg@15                       0.433447
ndcg@100                      0.433572
map@1                           0.8432
map@3                         0.882911
map@5                         0.901159
map@10                        0.909595
map@15                        0.915115
map@100                       0.915415
recall@1                      0.643945
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/Magic Square/hparams-userknn_7d_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-05-15,1,7d,2023-05-15,0.004682,0.200665,13,1,5.737452,0.791506,...,0.977888,1.000000,1.0,0.405994,0.722334,0.833833,0.913971,0.922746,0.922746,0.379332
1,2023-05-15,1,14d,2023-05-15,0.004439,0.198062,13,1,5.737452,0.791506,...,0.977888,1.000000,1.0,0.405994,0.722334,0.833833,0.913971,0.922746,0.922746,0.377642
2,2023-05-15,1,21d,2023-05-15,0.004588,0.200216,13,1,5.737452,0.791506,...,0.977888,1.000000,1.0,0.405994,0.722334,0.833833,0.913971,0.922746,0.922746,0.376745
3,2023-05-15,1,30d,2023-05-15,0.004562,0.202794,13,1,5.737452,0.791506,...,0.977888,1.000000,1.0,0.405994,0.722334,0.833833,0.913971,0.922746,0.922746,0.380458
4,2023-05-15,1,60d,2023-05-15,0.004486,0.198150,13,1,5.737452,0.791506,...,0.977888,1.000000,1.0,0.405994,0.722334,0.833833,0.913971,0.922746,0.922746,0.376502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2023-07-17,15,21d,2023-07-17,0.329599,2.822635,16,1,4.295276,0.837008,...,0.966252,0.994335,1.0,0.754315,0.876368,0.898441,0.934569,0.946005,0.947439,0.727027
766,2023-07-17,15,30d,2023-07-17,0.381243,3.203500,16,1,4.947955,0.824535,...,0.965064,0.995680,1.0,0.736893,0.849639,0.870331,0.908617,0.921766,0.923636,0.746834
767,2023-07-17,15,60d,2023-07-17,0.563180,4.314366,16,1,5.669910,0.713690,...,0.899606,0.994091,1.0,0.763759,0.785728,0.815473,0.860238,0.878637,0.881414,0.769621
768,2023-07-17,15,90d,2023-07-17,0.582203,4.474541,16,1,5.839371,0.722488,...,0.905302,0.988408,1.0,0.745326,0.781091,0.814616,0.864996,0.884393,0.887335,0.780650


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train   avg_recs  precision@5  precision@10  \
window_size k                                                      
7d          15    0.055863  15.296094     0.425822      0.381021   
            10    0.056005  15.296094     0.422747      0.377472   
            6     0.056141  15.296094     0.421010      0.378186   
            9     0.055837  15.296094     0.423121      0.376930   
            5     0.055990  15.296094     0.420569      0.377435   
...                    ...        ...          ...           ...   
90d         10    0.238355  16.498979     0.376663      0.349113   
60d         10    0.235563  16.479449     0.375567      0.348199   
10YE        15    0.236913  16.498979     0.371318      0.346914   
90d         15    0.237915  16.498979     0.371318      0.346914   
60d         15    0.235167  16.479449     0.370745      0.345895   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          15       0.064895  0.081935  0.089751  0.103084  0.567127   
            10       0.064895  0.081685  0.089434  0.102976  0.563262   
            6        0.064895  0.081849  0.089819  0.103168  0.562035   
            9        0.064895  0.081747  0.089440  0.103004  0.563189   
            5        0.064895  0.081854  0.089866  0.103198  0.561171   
...                       ...       ...       ...       ...       ...   
90d         10       0.070217  0.165394  0.184929  0.236652  0.462049   
60d         10       0.070120  0.164468  0.183924  0.235644  0.461607   
10YE        15       0.070217  0.163019  0.183044  0.235439  0.459649   
90d         15       0.070217  0.163019  0.183044  0.235439  0.459649   
60d         15       0.070120  0.162341  0.181986  0.234467  0.459582   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          15  0.600140  0.675478  0.506568   0.720752         1.0   
            10  0.595336  0.672130  0.508274   0.715696         1.0   
            6   0.595126  0.671126  0.507372   0.720252         1.0   
            9   0.594737  0.671965  0.508625   0.715217         1.0   
            5   0.593825  0.670683  0.507515   0.718371         1.0   
...                  ...       ...       ...        ...         ...   
90d         10  0.491401  0.584745  0.380762   0.584785         1.0   
60d         10  0.490963  0.584348  0.381315   0.584638         1.0   
10YE        15  0.488562  0.582333  0.376827   0.580242         1.0   
90d         15  0.488562  0.582333  0.376827   0.580242         1.0   
60d         15  0.488220  0.582043  0.378117   0.579634         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          15       0.577074        0.692427         0.776054  
            10       0.570270        0.680000         0.762622  
            6        0.563689        0.676925         0.760444  
            9        0.569093        0.681080         0.764125  
            5        0.562911        0.681703         0.764676  
...                       ...             ...              ...  
90d         10       0.524393        0.634304         0.738368  
60d         10       0.524341        0.633743         0.737740  
10YE        15       0.543754        0.637773         0.742739  
90d         15       0.543754        0.637773         0.742739  
60d         15       0.543871        0.637246         0.742144  

[77 rows x 17 columns]

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Magic Square/models/userknn-best-avg_7d_normalize.parquet


/tmp/ipykernel_483168/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-05-15,7d,15,2023-05-15,0.004444,0.200305,13,1,5.737452,0.756757,0.675676,0.549035,0.365251,...,0.977281,1.000000,1.0,0.400365,0.726437,0.834938,0.916829,0.925892,0.925892,0.379055
2023-05-22,7d,15,2023-05-22,0.006955,0.259174,32,1,20.579288,0.472492,0.446602,0.418770,0.383819,...,0.639278,0.783137,1.0,0.362944,0.452566,0.517166,0.617167,0.698032,0.757359,1.271537
2023-05-29,7d,15,2023-05-29,0.015458,0.562243,31,1,20.691438,0.560582,0.504577,0.487561,0.432149,...,0.754663,0.864524,1.0,0.323790,0.431896,0.521773,0.659423,0.713000,0.743795,3.039033
2023-06-05,7d,15,2023-06-05,0.038940,0.996213,25,1,15.005519,0.517660,0.470199,0.460044,0.434106,...,0.690772,0.861245,1.0,0.332293,0.449822,0.533708,0.659173,0.722619,0.741207,3.871934
2023-06-12,7d,15,2023-06-12,0.043011,1.221935,23,1,14.218667,0.504889,0.438519,0.407111,0.370667,...,0.644860,0.911845,1.0,0.303218,0.445408,0.529958,0.660475,0.718381,0.749999,3.551305
2023-06-19,7d,15,2023-06-19,0.055166,1.293237,19,1,12.411872,0.747945,0.617352,0.537534,0.422557,...,0.879973,0.967403,1.0,0.437610,0.627707,0.708097,0.821074,0.860214,0.874733,2.631589
2023-06-26,7d,15,2023-06-26,0.044722,1.387676,22,1,14.352895,0.364632,0.330203,0.376995,0.427308,...,0.701095,0.966591,1.0,0.306770,0.503519,0.409452,0.607732,0.642275,0.650981,5.430528
2023-07-03,7d,15,2023-07-03,0.152523,2.157078,30,1,18.978957,0.619396,0.565111,0.538701,0.464959,...,0.623443,0.783218,1.0,0.307281,0.389559,0.475857,0.605994,0.714088,0.790989,4.963924
2023-07-10,7d,15,2023-07-10,0.075907,4.203841,33,1,15.097138,0.420198,0.310310,0.276610,0.242185,...,0.633877,0.930770,1.0,0.569263,0.652517,0.667218,0.719902,0.750838,0.768004,8.616149


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Magic Square/models/userknn-best-val_7d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-05-15,1,14d,0.004439,5.737452,0.552896,0.364479,0.037954,0.235184,0.246356,0.248051,0.808283,0.835955,0.841177,0.814133,0.977888,1.0,0.833833,0.913971,0.922746
2023-05-22,1,14d,0.011496,19.220302,0.473434,0.442981,0.091404,0.122719,0.130995,0.156486,0.539308,0.557909,0.644716,0.386652,0.586863,1.0,0.431474,0.574529,0.764018
2023-05-29,15,7d,0.015458,20.691438,0.487561,0.432149,0.073134,0.054111,0.059052,0.066458,0.567227,0.604925,0.675781,0.521016,0.754663,1.0,0.521773,0.659423,0.743795
2023-06-05,15,14d,0.055463,14.086324,0.532299,0.485160,0.076634,0.072212,0.079152,0.090288,0.589718,0.630843,0.704023,0.464769,0.717374,1.0,0.541520,0.709525,0.802904
2023-06-12,4,14d,0.084695,13.761174,0.465777,0.413876,0.066858,0.109122,0.119853,0.141352,0.525186,0.553713,0.624726,0.425274,0.663807,1.0,0.543069,0.701509,0.790203
2023-06-19,15,7d,0.055166,12.411872,0.537534,0.422557,0.055744,0.155610,0.163597,0.170291,0.773147,0.802046,0.832703,0.695905,0.879973,1.0,0.708097,0.821074,0.874733
2023-06-26,1,14d,0.104651,12.206603,0.589795,0.480990,0.065473,0.085645,0.093114,0.104327,0.596065,0.632234,0.695081,0.496009,0.764126,1.0,0.530209,0.699247,0.750497
2023-07-03,10,7d,0.152444,18.978957,0.522598,0.468710,0.098618,0.079284,0.081420,0.094916,0.636081,0.631627,0.725718,0.462400,0.626334,1.0,0.467707,0.607962,0.790918
2023-07-10,1,7d,0.075363,15.097138,0.318217,0.252614,0.045482,0.116918,0.129257,0.156424,0.469785,0.494667,0.569699,0.528408,0.679708,1.0,0.610030,0.631421,0.674296


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000
mean,0.078447,14.506868,0.477584,0.403031,0.066476,0.135626,0.143068,0.157160,0.622010,0.646538,0.709763,0.544096,0.738766,1.0,0.584278,0.704857,0.793416
min,0.011496,4.108000,0.318217,0.228240,0.024936,0.054111,0.059052,0.066458,0.469785,0.494667,0.569699,0.386652,0.586863,1.0,0.431474,0.574529,0.674296
25%,0.055166,12.411872,0.465777,0.413876,0.055744,0.079284,0.081420,0.094916,0.539308,0.557909,0.644716,0.462400,0.663807,1.0,0.521773,0.631421,0.750497
50%,0.075363,14.086324,0.487561,0.432149,0.066858,0.109122,0.119853,0.141352,0.589718,0.630843,0.695081,0.496009,0.717374,1.0,0.541520,0.699247,0.790203
75%,0.104651,18.978957,0.532299,0.468710,0.076634,0.122719,0.130995,0.156486,0.636081,0.632234,0.725718,0.528408,0.764126,1.0,0.610030,0.709525,0.802904
max,0.152444,20.691438,0.589795,0.485160,0.098618,0.425013,0.431174,0.433896,0.901574,0.910880,0.915420,0.916431,0.976048,1.0,0.904621,0.939019,0.949382
std,0.051280,4.992031,0.085302,0.095692,0.022596,0.112751,0.112773,0.109756,0.134769,0.130384,0.106153,0.164562,0.123702,0.0,0.144009,0.113045,0.079375


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Magic Square/models/userknn-best-test_7d_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2023-05-22,1,14d,2023-05-22,0.011496,0.388412,32,1,19.220302,0.574514,0.493161,0.473434,0.442981,...,0.586863,0.758785,1.0,0.271712,0.355118,0.431474,0.574529,0.684370,0.764018,1.309404
2023-05-29,1,14d,2023-05-29,0.022937,0.703574,31,1,18.305741,0.463284,0.441923,0.437383,0.406409,...,0.652660,0.796294,1.0,0.302409,0.389274,0.457007,0.608648,0.665128,0.704040,3.046536
2023-06-05,15,7d,2023-06-05,0.038940,0.996213,25,1,15.005519,0.517660,0.470199,0.460044,0.434106,...,0.690772,0.861245,1.0,0.332293,0.449822,0.533708,0.659173,0.722619,0.741207,3.871934
2023-06-12,15,14d,2023-06-12,0.085004,2.070080,23,1,13.761174,0.530354,0.492773,0.464043,0.405270,...,0.638837,0.857062,1.0,0.262811,0.434090,0.547528,0.703929,0.762856,0.795980,3.696116
2023-06-19,4,14d,2023-06-19,0.103924,2.039709,19,1,11.799587,0.712810,0.623049,0.558953,0.452961,...,0.851920,0.960072,1.0,0.376058,0.575120,0.670159,0.806162,0.844678,0.857821,2.711178
2023-06-26,15,7d,2023-06-26,0.044722,1.387676,22,1,14.352895,0.364632,0.330203,0.376995,0.427308,...,0.701095,0.966591,1.0,0.306770,0.503519,0.409452,0.607732,0.642275,0.650981,5.430528
2023-07-03,1,14d,2023-07-03,0.205682,8.589808,30,1,19.149116,0.242314,0.372791,0.337981,0.311402,...,0.496326,0.641034,1.0,0.261935,0.286987,0.318607,0.390584,0.436765,0.468137,6.005307
2023-07-10,10,7d,2023-07-10,0.076263,4.157757,33,1,15.097138,0.422675,0.309760,0.274078,0.242020,...,0.634906,0.930713,1.0,0.566644,0.651352,0.681003,0.719229,0.750460,0.767263,8.890826
2023-07-17,1,7d,2023-07-17,0.070455,0.516469,16,1,6.329073,0.725240,0.430245,0.345048,0.257188,...,0.932982,0.992202,1.0,0.729833,0.765833,0.810705,0.875276,0.900329,0.903014,0.612136


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-06-19 00:00:00,0.073269,2.316633,25.666667,1.0,14.780061,0.505943,0.440456,0.414218,0.375516,...,0.687374,0.862666,1.0,0.378941,0.490124,0.539961,0.660585,0.712164,0.739162,3.952663
min,2023-05-22 00:00:00,0.011496,0.388412,16.000000,1.0,6.329073,0.242314,0.309760,0.274078,0.242020,...,0.496326,0.641034,1.0,0.261935,0.286987,0.318607,0.390584,0.436765,0.468137,0.612136
25%,2023-06-05 00:00:00,0.038940,0.703574,22.000000,1.0,13.761174,0.422675,0.372791,0.345048,0.311402,...,0.634906,0.796294,1.0,0.271712,0.389274,0.431474,0.607732,0.665128,0.704040,2.711178
50%,2023-06-19 00:00:00,0.070455,1.387676,25.000000,1.0,15.005519,0.517660,0.441923,0.437383,0.406409,...,0.652660,0.861245,1.0,0.306770,0.449822,0.533708,0.659173,0.722619,0.764018,3.696116
75%,2023-07-03 00:00:00,0.085004,2.070080,31.000000,1.0,18.305741,0.574514,0.492773,0.464043,0.434106,...,0.701095,0.960072,1.0,0.376058,0.575120,0.670159,0.719229,0.762856,0.795980,5.430528
max,2023-07-17 00:00:00,0.205682,8.589808,33.000000,1.0,19.220302,0.725240,0.623049,0.558953,0.452961,...,0.932982,0.992202,1.0,0.729833,0.765833,0.810705,0.875276,0.900329,0.903014,8.890826
std,NaN,0.058063,2.623142,6.123724,0.0,4.075984,0.155942,0.095957,0.087461,0.082490,...,0.132530,0.115135,0.0,0.162127,0.151591,0.156095,0.140713,0.132677,0.126507,2.533858
